# Create and Invoke Agent via Boto3 SDK

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use the `bedrock-agent` and the `bedrock-agent-runtime` boto3 clients to:
- create an agent
- create and action group
- associate the agent with the action group and prepare the agent
- create an agent alias
- invoke the agent

We will use Bedrock's Claude v2.1 using the Boto3 API. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Pre-requisites
This notebook requires permissions to: 
- create and delete Amazon IAM roles
- create, update and invoke AWS Lambda functions 
- create, update and delete Amazon S3 buckets 
- access Amazon Bedrock 

If you are running this notebook without an Admin role, make sure that your role include the following managed policies:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonBedrockFullAccess

And the following custom policy for Amazon OpenSearch Serverless access (required for Lab 7.2)

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "aoss:*",
            "Resource": "*"
        }
    ]
}
```

#### Context
We will demonstrate how to create and invoke an agent for Bedrock using the Boto3 SDK

#### Use case
For this notebook, we use an insurance claims use case to build our Agent. The agent helps the insurance provider checking the open claims, identifying the details for a specific claim, get open documents for a claim and send reminders for a claim policyholder. The following diagram illustrates the sample process flow.

![sequence-flow-agent](images/92-agent-workflow.png)

#### Architecture
The following diagram depicts a high-level architecture of this solution.

![architecture-diagram](images/92-agent-architecture.png)

The Agent created can handle the follow tasks:
- Get Open Claims
- Get Claim Details
- Get Claim Outstanding Documents
- Send Claim reminder

## Notebook setup
Before starting, let's import the required packages and configure the support variables

In [1]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint

In [2]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

[2024-06-05 08:22:23,019] p13432 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '630441275995')

In [5]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
#agent_name = "new-insurance-claims-agent"
#agent_name = "new-insurance-claims-agent-v2"
agent_name = "new-insurance-claims-agent-v3"
agent_alias_name = "workshop-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_key = f'{agent_name}-schema.json'
schema_name = 'insurance_claims_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
# agent_role_name = f'newAmazonBedrockExecutionRoleForAgents_{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgentsv2_{suffix}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{suffix}'

### Create S3 bucket and upload API Schema

Agents require an API Schema stored on s3. Let's create an S3 bucket to store the file and upload the file to the newly created bucket

In [6]:
# Create S3 bucket for Open API schema
s3bucket = s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={ 'LocationConstraint': region } 
)

In [7]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)

### Create Lambda function for Action Group
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [8]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'defc5eb5-803b-4823-890c-8e6db4a5cc67',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:22:41 GMT',
   'x-amzn-requestid': 'defc5eb5-803b-4823-890c-8e6db4a5cc67',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

Take a look at the Lambda function code that will be used as an Action group for the agent

In [9]:
!pygmentize lambda_function.py

#!/usr/bin/env python3
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
import json


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_named_property(event, name):
    return next(
        item for item in
        event['requestBody']['content']['application/json']['properties']
        if item['name'] == name)['value']


def claim_detail(payload):
    claim_id = payload['parameters'][0]['value']
    if claim_id == 'claim-857':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "21-Jul-2023",
                "lastActivityDate": "25-Jul-2023",
                "status": "Open",
                "policyType": "Vehicle"
            }
        }
    elif claim_id == 'claim-006':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "20-May-2

In [10]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

### Create Agent
We will now create our agent. To do so, we first need to create the agent policies that allow bedrock model invocation  and s3 bucket access. 

In [11]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-v2:1"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



Next, we will create a policy document that allows fetching of the Agent's OpenAPI schema from S3:

In [12]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

Finally, create a role with the above two policies attached

In [13]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'd9e51727-805c-4fcf-8584-4532e091a476',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:23:11 GMT',
   'x-amzn-requestid': 'd9e51727-805c-4fcf-8584-4532e091a476',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Creating Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [14]:
# Create Agent
agent_instruction = """
You are an agent that can handle various tasks related to insurance claims, including looking up claim 
details, finding what paperwork is outstanding, and sending reminders. Only send reminders if you have been 
explicitly requested to do so. If an user asks about your functionality, provide guidance in natural language 
and do not include function names on the output."""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for handling insurance claims.",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2:1",
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [15]:
response

{'ResponseMetadata': {'RequestId': 'a25e37ad-57bc-4274-8dc6-70339aba4361',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:23:12 GMT',
   'content-type': 'application/json',
   'content-length': '899',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a25e37ad-57bc-4274-8dc6-70339aba4361',
   'x-amz-apigw-id': 'Y4oBkFRLPHcEp4w=',
   'x-amzn-trace-id': 'Root=1-66602070-76efc36c511a91ca0c42277c'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:630441275995:agent/MEEVWNIW1Y',
  'agentId': 'MEEVWNIW1Y',
  'agentName': 'new-insurance-claims-agent-v3',
  'agentResourceRoleArn': 'arn:aws:iam::630441275995:role/AmazonBedrockExecutionRoleForAgentsv2_us-west-2-630441275995',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 6, 5, 8, 23, 12, 291688, tzinfo=tzlocal()),
  'description': 'Agent for handling insurance claims.',
  'foundationModel': 'anthropic.claude-v2:1',
  'idleSessionTTLInSeconds': 1800,
  'instruction': '\nY

Let's now store the agent id in a local variable to use it on the next steps

In [16]:
agent_id = response['agent']['agentId']
agent_id

'MEEVWNIW1Y'

### Create Agent Action Group
We will now create and agent action group that uses the lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [17]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='ClaimsActionGroupv2',
    #actionGroupName='ClaimManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for listing claims, identifying missing paperwork, sending reminders'
)

In [18]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '8623f405-7b49-4549-bef7-0291089b5324',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:24:00 GMT',
   'content-type': 'application/json',
   'content-length': '633',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8623f405-7b49-4549-bef7-0291089b5324',
   'x-amz-apigw-id': 'Y4oJBFArvHcEHmg=',
   'x-amzn-trace-id': 'Root=1-6660209f-47eb49896460f1c335ae0576'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:630441275995:function:new-insurance-claims-agent-v3-us-west-2-630441275995'},
  'actionGroupId': 'VN1CMRNLS5',
  'actionGroupName': 'ClaimsActionGroupv2',
  'actionGroupState': 'ENABLED',
  'agentId': 'MEEVWNIW1Y',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'new-insurance-claims-agent-v3-us-west-2-630441275995',
    's3ObjectKey': 'new-insurance-claims-agent-v3-schema.json'}},
  'createdAt': datetime.datetime(2024, 6, 5, 8, 24, 0, 324210, t

### Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [19]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Preparing Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [20]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '6123a201-f358-4aab-b109-647de0051a66',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:24:11 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6123a201-f358-4aab-b109-647de0051a66',
   'x-amz-apigw-id': 'Y4oK5HTcvHcEH1g=',
   'x-amzn-trace-id': 'Root=1-666020ab-175a6b2c0732b6b572955071'},
  'RetryAttempts': 0},
 'agentId': 'MEEVWNIW1Y',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 6, 5, 8, 24, 11, 876051, tzinfo=tzlocal())}

### Create Agent alias
We will now create an alias of the agent that can be used to deploy the agent.

In [21]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [22]:
agent_alias

{'ResponseMetadata': {'RequestId': 'd35c0611-0b05-443f-85fb-164e59666809',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:24:46 GMT',
   'content-type': 'application/json',
   'content-length': '340',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd35c0611-0b05-443f-85fb-164e59666809',
   'x-amz-apigw-id': 'Y4oQUEUivHcELZg=',
   'x-amzn-trace-id': 'Root=1-666020ce-7175d6ab14f6879a269dec65'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-west-2:630441275995:agent-alias/MEEVWNIW1Y/BGFI8PN1AC',
  'agentAliasId': 'BGFI8PN1AC',
  'agentAliasName': 'workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'MEEVWNIW1Y',
  'createdAt': datetime.datetime(2024, 6, 5, 8, 24, 46, 659920, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 6, 5, 8, 24, 46, 659920, tzinfo=tzlocal())}}

### Invoke Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [23]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Tell me about claim-857",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-06-05 08:24:51,700] p13432 {3818358944.py:21} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 05 Jun 2024 08:24:51 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '147371b0-2315-11ef-98ce-0ae2b1fad1a9',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '1c099b81-4b3a-44f6-a8e6-0f290bc52378'},
                      'HTTPStatusCode': 200,
                      'RequestId': '1c099b81-4b3a-44f6-a8e6-0f290bc52378',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f106c78b010>,
 'contentType': 'application/json',
 'sessionId': '147371b0-2315-11ef-98ce-0ae2b1fad1a9'}


In [24]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-06-05 08:24:58,087] p13432 {<timed exec>:11} INFO - {
  "agentAliasId": "BGFI8PN1AC",
  "agentId": "MEEVWNIW1Y",
  "agentVersion": "1",
  "sessionId": "147371b0-2315-11ef-98ce-0ae2b1fad1a9",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "\n\nHuman:"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<tools>\n    <tool_description>\n<tool_name>GET::ClaimsActionGro

CPU times: user 19 ms, sys: 2.63 ms, total: 21.7 ms
Wall time: 10.9 s


In [25]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

Here are the details for claim-857:

Claim ID: claim-857
Created Date: 21-Jul-2023  
Last Activity Date: 25-Jul-2023
Status: Open
Policy Type: Vehicle


### Clean up (optional)
The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:
1. update the action group to disable it
2. delete agent action group
3. delete agent alias
4. delete agent
5. delete lambda function
6. empty created s3 bucket
7. delete s3 bucket

In [26]:
 # This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first

action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [27]:
 agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [28]:
 agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [29]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_name
)

{'ResponseMetadata': {'RequestId': '3b0a30c0-9e23-4b72-b64d-ccf2d773d0a4',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 08:25:16 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3b0a30c0-9e23-4b72-b64d-ccf2d773d0a4'},
  'RetryAttempts': 0}}

In [30]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'J3KMYRHD6XKRSZ3D',
  'HostId': 'o3/+YFMdzGnTHWEk31RCibfdtzQp25tPLm1aybjeRZQP7hQ/HGuHKV4k6qjw5Y+sDCzL67rW0tHXDgsP6oZgvA==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'o3/+YFMdzGnTHWEk31RCibfdtzQp25tPLm1aybjeRZQP7hQ/HGuHKV4k6qjw5Y+sDCzL67rW0tHXDgsP6oZgvA==',
   'x-amz-request-id': 'J3KMYRHD6XKRSZ3D',
   'date': 'Wed, 05 Jun 2024 08:25:17 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [31]:
# Delete IAM Roles and policies

for policy in [bedrock_agent_bedrock_allow_policy_name, bedrock_agent_s3_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy, agent_s3_schema_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You